### Importing necessary libraries
- **glob** and **os**: To handle file operations.
- **torch**: For tensor operations and managing model computations on GPU.
- **Path**: From `pathlib` for easy file path manipulation.
- **transformers**: Importing `AutoTokenizer` for tokenization, `BertForMaskedLM` for the BERT model, `Trainer` and `TrainingArguments` for setting up the training pipeline.
- **datasets**: To handle dataset creation.
- **DataCollatorForLanguageModeling**: To handle token masking for MLM (Masked Language Modeling).


In [4]:
# Import necessary libraries
import glob
import os
import torch
from pathlib import Path
from transformers import AutoTokenizer, BertForMaskedLM, Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling

### Set up device for computation
- **Check for GPU availability**: The script sets the device to `cuda` if a GPU is available, otherwise it defaults to `cpu`.
- **Disable Weights & Biases (WANDB) logging**: To prevent logging through WANDB during the training process.


In [5]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Disable WANDB logging
os.environ["WANDB_DISABLED"] = "true"

Using device: cuda


In [9]:
paths = {
    "culture": 'nlp_dataset/articles-culture/*.txt',
    "economy": 'nlp_dataset/articles-economy/*.txt',
    "international": 'nlp_dataset/articles-international/*.txt',
    "local": 'nlp_dataset/articles-local/*.txt',
    "religion": 'nlp_dataset/articles-religion/*.txt',
    "sports": 'nlp_dataset/articlesSports/*.txt'
}

In [ ]:
print(len(paths.values()))

6


### Data Loading Functions
- **`load_data(category)`**: Loads text files from a specific category using the provided file paths, reading and appending their content into a list.
- **`load_all_data()`**: Iterates through all categories, loading data from each one and combining them into a single list.
- **Load articles**: After calling `load_all_data()`, the total number of loaded articles is printed for verification.


In [12]:
# Function to load data from a specific category
def load_data(category):
    files = glob.glob(paths[category])
    articles = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            articles.append(f.read())
    return articles

# Function to load data from all categories
def load_all_data():
    all_articles = []
    for category in paths.keys():
        all_articles.extend(load_data(category))
    return all_articles

# Load articles from all categories
all_articles = load_all_data()
print(f"Total number of articles loaded: {len(all_articles)}")

Total number of articles loaded: 4095


### Tokenization with AraBERT
- **Initialize AraBERT tokenizer**: Uses the pre-trained tokenizer from "aubmindlab/bert-base-arabertv02" to handle Arabic text.
- **`tokenize_articles(articles)`**: Tokenizes the input articles, applies padding to ensure uniform length, truncates articles longer than 512 tokens, and converts them to PyTorch tensors.
- **Tokenize all articles**: The entire dataset is tokenized, and a confirmation message is printed once the process is complete.


In [13]:
# Initialize AraBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

# Function to tokenize articles
def tokenize_articles(articles):
    return tokenizer(articles, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize all articles
tokenized_all_articles = tokenize_articles(all_articles)
print("All articles tokenized.")

All articles tokenized.


### Splitting Data into Training and Evaluation Sets
- **`split_dataset(tokenized_articles)`**: 
  - Splits the tokenized articles into **90% training** and **10% evaluation** sets.
  - Separates both the `input_ids` and `attention_mask` for the training and evaluation datasets.
- **Convert to Dataset objects**: 
  - `train_dataset` and `eval_dataset` are created from the split data for compatibility with the Hugging Face `Trainer`.
- **Dataset size**: Prints the size of both the training and evaluation datasets to confirm the split.


In [14]:
# Function to split the tokenized articles into train and eval sets
def split_dataset(tokenized_articles):
    train_size = int(0.9 * len(tokenized_articles['input_ids']))
    train_dataset = {
        "input_ids": tokenized_articles['input_ids'][:train_size],
        "attention_mask": tokenized_articles['attention_mask'][:train_size],
    }
    eval_dataset = {
        "input_ids": tokenized_articles['input_ids'][train_size:],
        "attention_mask": tokenized_articles['attention_mask'][train_size:],
    }
    return train_dataset, eval_dataset

# Split the data
train_data, eval_data = split_dataset(tokenized_all_articles)

# Convert data to Dataset objects
train_dataset = Dataset.from_dict(train_data)
eval_dataset = Dataset.from_dict(eval_data)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

Training dataset size: 3685
Evaluation dataset size: 410


### Loading the AraBERT Model for Masked Language Modeling (MLM)
- **BertForMaskedLM**: Loads the pre-trained AraBERT model (`bert-base-arabertv02`) specifically for Masked Language Modeling.
- **Move model to GPU**: The model is transferred to GPU if available for faster training.

### Data Collator for MLM
- **DataCollatorForLanguageModeling**: Dynamically masks 15% of tokens (`mlm_probability=0.15`) during training, enabling the MLM objective where the model learns to predict the masked tokens.

### Training Arguments
- **`output_dir`**: Directory to store the results.
- **Batch size**: Set to 8 per device (adjustable based on GPU capacity).
- **Training epochs**: Set to 10 epochs.
- **Save strategy**: Saves the model every 10,000 steps and limits to 2 checkpoints.
- **Disable wandb logging**: Ensures that Weights & Biases logging is turned off.

Model and training arguments are now ready for training.


In [18]:
# Load BERT model for masked language modeling
model = BertForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02")
model.to(device)  # Move model to GPU

# Data collator for masked language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Adjust based on GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_steps=10_000,
    save_total_limit=2,
    report_to=[]  # Disable wandb logging
)

print("Model loaded and training arguments set.")

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

### Initialize the Trainer
- **Trainer**: Initializes the Hugging Face `Trainer` with the following components:
  - **model**: AraBERT model for Masked Language Modeling (MLM).
  - **args**: Training arguments defined earlier (batch size, epochs, evaluation strategy, etc.).
  - **data_collator**: Dynamically masks tokens during training for MLM.
  - **train_dataset & eval_dataset**: The training and evaluation datasets previously split and tokenized.

### Start Training
- **trainer.train()**: Begins the training process for the model with the specified datasets and arguments.


In [45]:
# Initialize the Trainer with both train and eval datasets
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Start training
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.843200,1.576623
2,1.737400,1.542854
3,1.670100,1.537364
4,1.607600,1.517988
5,1.555000,1.511468
6,1.502600,1.501382
7,1.473600,1.484621
8,1.437900,1.479608
9,1.416600,1.471871
10,1.416500,1.468602


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=10270, training_loss=1.5718477629872352, metrics={'train_runtime': 14438.1164, 'train_samples_per_second': 11.38, 'train_steps_per_second': 0.711, 'total_flos': 4.32614480805888e+16, 'train_loss': 1.5718477629872352, 'epoch': 10.0})

In [46]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 1.4493675231933594, 'eval_runtime': 59.8225, 'eval_samples_per_second': 30.524, 'eval_steps_per_second': 1.922, 'epoch': 10.0}


In [47]:
# Save the model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")
print("Model and tokenizer saved.")

Model and tokenizer saved.


In [ ]:
# Load the fine-tuned model and tokenizer
model = BertForMaskedLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")

# Move model to GPU if available
model.to(device)



### Function for Masked Token Prediction and Text Completion
- **`mask_and_complete_text()`**: This function iterates over an input text, predicting and replacing each `[MASK]` token using the AraBERT model.
  - **Tokenization**: The input text is tokenized and sent to the model for prediction.
  - **Predict masked tokens**: For each `[MASK]` token found, the model predicts the most likely token.
  - **Iterative prediction**: The function continuously replaces the first `[MASK]` in the text with the predicted token and repeats until all masks are replaced.
  - **GPU support**: Moves the inputs to GPU if available for faster computation.

### Example usage
- **Input**: The example sentence includes `[MASK]` tokens that the model will attempt to complete.
- **Output**: The completed sentence is printed with the predicted tokens replacing the masks.

In this example, AraBERT predicts the missing words in the sentence:  
*"عاصمة [MASK] هي الرياض وجدة تقع على البحر [MASK]"*.


In [89]:
def mask_and_complete_text(input_text, mask_token="[MASK]"):
    """Uses AraBERT to predict missing tokens in Arabic text using masked language modeling."""
    
    while mask_token in input_text:
        # Tokenize the input text
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
        
        # Move inputs to GPU if available
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Run the model to predict the masked tokens
        with torch.no_grad():
            outputs = model(**inputs)

        # Get the predicted token ids for the [MASK] position
        mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]

        # Check if there are still [MASK] tokens to predict
        if len(mask_token_index) == 0:
            break

        # Get the prediction for the first [MASK] token found
        predicted_token_id = outputs.logits[0, mask_token_index[0]].argmax(axis=-1)

        # Decode the predicted token
        predicted_token = tokenizer.decode(predicted_token_id)

        # Replace the first [MASK] token with the predicted token in the original text
        input_text = input_text.replace(mask_token, "[" + predicted_token +"]", 1)

    return input_text

# Example usage for text completion with Arabic input
input_text = "عاصمة [MASK]  هي الرياض وجدة تقع على البحر [MASK]"
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text: {completed_text}")


Completed Text: عاصمة [السعودية]  هي الرياض وجدة تقع على البحر [الأحمر]


# TEST:


# Masked Text:

1. الرياضة:

الرياضة تعتبر جزءًا مهمًا من حياة الأفراد والمجتمعات. تساعد الرياضة في [MASK] اللياقة البدنية وتعزيز الصحة النفسية. في الآونة الأخيرة، أصبحت البطولات الرياضية الكبرى مثل [MASK] تحظى بمتابعة كبيرة على مستوى العالم.

2. الأخبار المحلية في عمان:

شهدت سلطنة عمان في الفترة الأخيرة تطورات ملحوظة في مجال البنية التحتية. المشاريع الجديدة تهدف إلى تحسين [MASK] وتعزيز الاقتصاد الوطني. بالإضافة إلى ذلك، يجري العمل على تطوير [MASK] السياحية لجذب المزيد من الزوار.

3. الأخبار العالمية:

تشهد الساحة العالمية تغيرات سريعة في العلاقات الدبلوماسية بين الدول. في الآونة الأخيرة، أعلنت [MASK] عن فرض عقوبات اقتصادية على [MASK]، مما أثار توترات سياسية كبيرة. بالإضافة إلى ذلك، تزايدت المخاوف بشأن استقرار [MASK] بسبب النزاعات المستمرة في المنطقة. من ناحية أخرى، تسعى [MASK] إلى تعزيز التعاون الدولي لمواجهة التحديات المشتركة مثل التغير المناخي.

4. الثقافة العمانية:

تعتبر الثقافة العمانية غنية بالتقاليد والفنون التي تمتد عبر قرون. تشتهر عمان بالفنون التقليدية مثل [MASK] والحرف اليدوية. كما تلعب الفعاليات الثقافية دورًا مهمًا في الحفاظ على [MASK] وتعريف الأجيال الجديدة بتاريخ البلاد.

5. الدين الإسلامي:

الدين الإسلامي يدعو إلى السلم والتعايش بين البشر. يؤكد الإسلام على أهمية [MASK] والتعاون بين أفراد المجتمع. كما يدعو إلى الالتزام بالأخلاق الحسنة مثل [MASK] والصدق في التعامل.

6. الاقتصاد:

الاقتصاد العالمي يشهد حاليًا تقلبات بسبب الأزمات المتلاحقة. في بعض الدول، تؤثر السياسات المالية على [MASK] المحلي بشكل مباشر. بينما تسعى الحكومات إلى تحقيق استقرار اقتصادي عبر تعزيز [MASK] وفتح أسواق جديدة.

# Unmasked text:

1. الرياضة:

الرياضة تعتبر جزءًا مهمًا من حياة الأفراد والمجتمعات. تساعد الرياضة في تحسين اللياقة البدنية وتعزيز الصحة النفسية. في الآونة الأخيرة، أصبحت البطولات الرياضية الكبرى مثل كأس العالم تحظى بمتابعة كبيرة على مستوى العالم.

2. الأخبار المحلية في عمان:

شهدت سلطنة عمان في الفترة الأخيرة تطورات ملحوظة في مجال البنية التحتية. المشاريع الجديدة تهدف إلى تحسين الخدمات وتعزيز الاقتصاد الوطني. بالإضافة إلى ذلك، يجري العمل على تطوير المواقع السياحية لجذب المزيد من الزوار.

3. الأخبار العالمية:

في الأخبار العالمية، تتصدر التطورات السياسية والاقتصادية المشهد في العديد من الدول. الأزمة في أوكرانيا تؤثر بشكل مباشر على الأسواقتشهد الساحة العالمية تغيرات سريعة في العلاقات الدبلوماسية بين الدول. في الآونة الأخيرة، أعلنت الولايات المتحدة عن فرض عقوبات اقتصادية على روسيا، مما أثار توترات سياسية كبيرة. بالإضافة إلى ذلك، تزايدت المخاوف بشأن استقرار الشرق الأوسط بسبب النزاعات المستمرة في المنطقة. من ناحية أخرى، تسعى الاتحاد الأوروبي إلى تعزيز التعاون الدولي لمواجهة التحديات المشتركة مثل التغير المناخي

4. الثقافة العمانية:

تعتبر الثقافة العمانية غنية بالتقاليد والفنون التي تمتد عبر قرون. تشتهر عمان بالفنون التقليدية مثل الموسيقى الشعبية والحرف اليدوية. كما تلعب الفعاليات الثقافية دورًا مهمًا في الحفاظ على التراث وتعريف الأجيال الجديدة بتاريخ البلاد.

5. الدين الإسلامي:

الدين الإسلامي يدعو إلى السلم والتعايش بين البشر. يؤكد الإسلام على أهمية التعاون والتعاون بين أفراد المجتمع. كما يدعو إلى الالتزام بالأخلاق الحسنة مثل الأمانة والصدق في التعامل.

6. الاقتصاد:

الاقتصاد العالمي يشهد حاليًا تقلبات بسبب الأزمات المتلاحقة. في بعض الدول، تؤثر السياسات المالية على الناتج المحلي بشكل مباشر. بينما تسعى الحكومات إلى تحقيق استقرار اقتصادي عبر تعزيز الاستثمار وفتح أسواق جديدة.

In [99]:
input_text = "الرياضة تعتبر جزءًا مهمًا من حياة الأفراد والمجتمعات. تساعد الرياضة في [MASK] اللياقة البدنية وتعزيز الصحة النفسية. في الآونة الأخيرة، أصبحت البطولات الرياضية الكبرى مثل [MASK] [MASK] تحظى بمتابعة كبيرة على مستوى العالم."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
الرياضة تعتبر جزءًا مهمًا من حياة الأفراد والمجتمعات. تساعد الرياضة في [زيادة] اللياقة البدنية وتعزيز الصحة النفسية. في الآونة الأخيرة، أصبحت البطولات الرياضية الكبرى مثل [كرة] [القدم] تحظى بمتابعة كبيرة على مستوى العالم.


In [91]:
input_text = "شهدت سلطنة عمان في الفترة الأخيرة تطورات ملحوظة في مجال البنية التحتية. المشاريع الجديدة تهدف إلى تحسين [MASK] وتعزيز الاقتصاد الوطني. بالإضافة إلى ذلك، يجري العمل على تطوير [MASK] السياحية لجذب المزيد من الزوار."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
شهدت سلطنة عمان في الفترة الأخيرة تطورات ملحوظة في مجال البنية التحتية. المشاريع الجديدة تهدف إلى تحسين [الخدمات] وتعزيز الاقتصاد الوطني. بالإضافة إلى ذلك، يجري العمل على تطوير [المرافق] السياحية لجذب المزيد من الزوار.


In [98]:
input_text = "تشهد الساحة العالمية تغيرات سريعة في العلاقات الدبلوماسية بين الدول. في الآونة الأخيرة، أعلنت [MASK] عن فرض عقوبات اقتصادية على [MASK]، مما أثار توترات سياسية كبيرة. بالإضافة إلى ذلك، تزايدت المخاوف بشأن استقرار [MASK] بسبب النزاعات المستمرة في المنطقة. من ناحية أخرى، تسعى [MASK] إلى تعزيز التعاون الدولي لمواجهة التحديات المشتركة مثل التغير المناخي."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
تشهد الساحة العالمية تغيرات سريعة في العلاقات الدبلوماسية بين الدول. في الآونة الأخيرة، أعلنت [واشنطن] عن فرض عقوبات اقتصادية على [إيران]، مما أثار توترات سياسية كبيرة. بالإضافة إلى ذلك، تزايدت المخاوف بشأن استقرار [العالم] بسبب النزاعات المستمرة في المنطقة. من ناحية أخرى، تسعى [الدول] إلى تعزيز التعاون الدولي لمواجهة التحديات المشتركة مثل التغير المناخي.


In [93]:
input_text = "تعتبر الثقافة العمانية غنية بالتقاليد والفنون التي تمتد عبر قرون. تشتهر عمان بالفنون التقليدية مثل [MASK] والحرف اليدوية. كما تلعب الفعاليات الثقافية دورًا مهمًا في الحفاظ على [MASK] وتعريف الأجيال الجديدة بتاريخ البلاد."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
تعتبر الثقافة العمانية غنية بالتقاليد والفنون التي تمتد عبر قرون. تشتهر عمان بالفنون التقليدية مثل [الفنون] والحرف اليدوية. كما تلعب الفعاليات الثقافية دورًا مهمًا في الحفاظ على [التراث] وتعريف الأجيال الجديدة بتاريخ البلاد.


In [95]:
input_text = "الدين الإسلامي يدعو إلى السلم والتعايش بين البشر. يؤكد الإسلام على أهمية [MASK] والتعاون بين أفراد المجتمع. كما يدعو إلى الالتزام بالأخلاق الحسنة مثل [MASK] والصدق في التعامل."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
الدين الإسلامي يدعو إلى السلم والتعايش بين البشر. يؤكد الإسلام على أهمية [التكافل] والتعاون بين أفراد المجتمع. كما يدعو إلى الالتزام بالأخلاق الحسنة مثل [الأمانة] والصدق في التعامل.


In [97]:
input_text = "الاقتصاد العالمي يشهد حاليًا تقلبات بسبب الأزمات المتلاحقة. في بعض الدول، تؤثر السياسات المالية على [MASK] المحلي بشكل مباشر. بينما تسعى الحكومات إلى تحقيق استقرار اقتصادي عبر تعزيز [MASK] وفتح أسواق جديدة."
completed_text = mask_and_complete_text(input_text)
print(f"Completed Text:\n{completed_text}")

Completed Text:
الاقتصاد العالمي يشهد حاليًا تقلبات بسبب الأزمات المتلاحقة. في بعض الدول، تؤثر السياسات المالية على [الناتج] المحلي بشكل مباشر. بينما تسعى الحكومات إلى تحقيق استقرار اقتصادي عبر تعزيز [صادراتها] وفتح أسواق جديدة.
